In [21]:
#%pip install mamba
#%pip install yfinance==0.1.67
#%pip install beautifulsoup4==4.10.0
#%pip install nbformat==4.2.0
#%pip install jupyter

import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup
import requests
import pytz
import plotly.graph_objects as go
from plotly.subplots import make_subplots



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



n

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [23]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [24]:
ticker = "TSLA"
tesla_stock = yf.Ticker(ticker)
#print(tesla_stock)

In [25]:
tesla_data = tesla_stock.history(period="max")


In [26]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

Date      Open      High       Low     Close     Volume  Dividends  \
0 2010-06-29  1.266667  1.666667  1.169333  1.592667  281494500          0   
1 2010-06-30  1.719333  2.028000  1.553333  1.588667  257806500          0   
2 2010-07-01  1.666667  1.728000  1.351333  1.464000  123282000          0   
3 2010-07-02  1.533333  1.540000  1.247333  1.280000   77097000          0   
4 2010-07-06  1.333333  1.333333  1.055333  1.074000  103003500          0   

   Stock Splits  
0           0.0  
1           0.0  
2           0.0  
3           0.0  
4           0.0

In [27]:
# get the webscrapping data
url = " https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text


In [28]:
# passing the data using Beautiful Soup
soup = BeautifulSoup(html_data, "html.parser")

In [29]:
tesla_revenue = pd.DataFrame(columns=['Date','Revenue'])

for row in soup.find("table", class_="historical_data_table table").find("tbody").find_all("tr"):
    # Find all cells within the row
    cells = row.find_all("td")
    date = cells[0].text.strip() 
    revenue = cells[1].text.strip() 
    # Append data to the DataFrame
    tesla_revenue = tesla_revenue.append({'Date': date, 'Revenue': revenue}, ignore_index=True)
    
#print(tesla_revenue.head())

In [30]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

In [31]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [32]:
tesla_revenue.tail()

Date Revenue
8   2013    2013
9   2012     413
10  2011     204
11  2010     117
12  2009     112

In [33]:
ticker = "GME"
game_stop = yf.Ticker(ticker)

In [34]:
gme_data = game_stop.history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()

Date      Open      High       Low     Close    Volume  Dividends  \
0 2002-02-13  1.620128  1.693350  1.603296  1.691667  76216000        0.0   
1 2002-02-14  1.712707  1.716073  1.670626  1.683250  11021600        0.0   
2 2002-02-15  1.683250  1.687458  1.658001  1.674834   8389600        0.0   
3 2002-02-19  1.666418  1.666418  1.578047  1.607504   7410400        0.0   
4 2002-02-20  1.615921  1.662210  1.603296  1.662210   6892800        0.0   

   Stock Splits  
0           0.0  
1           0.0  
2           0.0  
3           0.0  
4           0.0

In [35]:
url2 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data = requests.get(url2).text



In [36]:

soup = BeautifulSoup(html_data,"html.parser")

In [37]:
gme_revenue = pd.DataFrame(columns=['Date','Revenue'])

for row in soup.find("table", class_="historical_data_table table").find("tbody").find_all("tr"):
    # Find all cells within the row
    cells = row.find_all("td")
    date = cells[0].text.strip() 
    revenue = cells[1].text.strip() 
    # Append data to the DataFrame
    gme_revenue = gme_revenue.append({'Date': date, 'Revenue': revenue}, ignore_index=True)

gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"")
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

In [38]:
gme_revenue.tail()

Date Revenue
11  2009    8806
12  2008    7094
13  2007    5319
14  2006    3092
15  2005    1843

In [39]:
make_graph(tesla_data,tesla_revenue,'Tesla Graph')

In [40]:
make_graph(gme_data,gme_revenue,'Gamestop Graph')